# Clean up
❗ This notebook removes all resources created by a data transformation and ingestion project. The following code cells will:
- permanently delete project or projects you provisioned in your Studio environment
- permanently delete feature group or groups
- permanently delete project-provisioned S3 buckets
- permanently delete objects in S3 buckets under project-related prefixes

<div class="alert alert-info"> 💡 <strong> This notebook will delete resources in your AWS account. Please double check the names of resources to be deleted! </strong>
</div>

In [ ]:
import sagemaker
import boto3
import time
import json
import os

In [ ]:
# load environment variables from %store
%store -r 

In [ ]:
%store

In [ ]:
try:
    s3_data_prefix
    s3_flow_prefix
    s3_input_data_prefix
    s3_fs_query_output_prefix
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN 00-setup.ipynb and 01-feature-store-ingest-pipeline notebooks")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
sm = boto3.client("sagemaker")
s3 = boto3.resource('s3')

## Delete projects

In [ ]:
# Get all projects created by the current domain
projects = [
    {"ProjectName":p["ProjectName"], "ProjectId":p["ProjectId"]} for p in sm.list_projects(MaxResults=100, SortBy="CreationTime")["ProjectSummaryList"] 
        if sm.describe_project(ProjectName=p["ProjectName"])["CreatedBy"]["DomainId"] == domain_id and p["ProjectStatus"] == "CreateCompleted"
]

print(f"These projects have been created by domain {domain_id}: {json.dumps(projects, indent=2)}")

In [ ]:
# Select projects to be deleted
projects_to_delete = []

for p in projects:
    print(f"Are you sure you want to delete this project: {p['ProjectName']}? (y/n)")
    choice = input()
    if choice == 'y':
        projects_to_delete.append(p)
        
print(f"***************************************")
print(f"The following projects will be deleted:\n{json.dumps(projects_to_delete, indent=2)}")
print(f"***************************************")

<div class="alert alert-info"> 💡 <strong> The following code cell will delete all selected projects. All project CodeCommit code repositories and CI/CD CodePipeline pipelines will be deleted! </strong>
</div>

In [ ]:
for p in projects_to_delete:
    try:
        print(f"Deleting project {p['ProjectName']}:{sm.delete_project(ProjectName=p['ProjectName'])}")
    except Exception:
        pass

## Delete feature groups

In [ ]:
feature_groups = sm.list_feature_groups(
    FeatureGroupStatusEquals="Created", 
    SortOrder="Descending", 
    SortBy="CreationTime"
)["FeatureGroupSummaries"]

In [ ]:
feature_groups

In [ ]:
# Select feature groups to be deleted
feature_groups_to_delete = []

for fg in feature_groups:
    print(f"Are you sure you want to delete this feature group: {fg['FeatureGroupName']}? (y/n)")
    choice = input()
    if choice == 'y':
        feature_groups_to_delete.append(fg["FeatureGroupName"])
        
print(f"********************************************")
print(f"The following feature groups will be deleted:\n{json.dumps(feature_groups_to_delete, indent=2)}")
print(f"********************************************")


In [ ]:
def delete_offline_store(feature_group_name: str):
    try:
        offline_store_config = sm.describe_feature_group(FeatureGroupName=feature_group_name)['OfflineStoreConfig']

    except Exception:
        print(f'Feature group: {feature_group_name} does NOT have an offline store!')
        return
    
    offline_store_s3_uri = offline_store_config['S3StorageConfig']['ResolvedOutputS3Uri']
    print(f"all feature store objects under {offline_store_s3_uri} will be deleted!")
    print("Are you sure you want to these objects ? (y/n)")
    
    choice = input()
    if choice == 'y':
        !aws s3 rm {offline_store_s3_uri} --recursive

<div class="alert alert-info"> 💡 <strong> The following code cell will delete the selected feature groups!</strong>
</div>

In [ ]:
for fg in feature_groups_to_delete:
    print(f"Deleting the feature group: {fg}")
    delete_offline_store(fg)
    sm.delete_feature_group(FeatureGroupName=fg)

## Delete project-provisioned S3 buckets

<div class="alert alert-info"> 💡 <strong> The following code cell will delete all S3 buckets created by a project!</strong>
</div>

In [ ]:
print(f"*****************************************************")
print(f"The following S3 buckets will be removed permanently!")
print(f"*****************************************************")
for p in projects_to_delete:
    print(f"sagemaker-cp-{p['ProjectName']}-{p['ProjectId']}")
    print(f"sagemaker-ct-{p['ProjectName']}-{p['ProjectId']}")

In [ ]:
for p in projects_to_delete:
    !aws s3 rb s3://sagemaker-cp-{p['ProjectName']}-{p['ProjectId']} --force    
    !aws s3 rb s3://sagemaker-ct-{p['ProjectName']}-{p['ProjectId']} --force

## Remove project-related objects from S3 data bucket

<div class="alert alert-info"> 💡 <strong> The following code cells will delete all objects under specified S3 prefixes!</strong>
</div>

In [ ]:
prefixes_to_delete = [
    s3_data_prefix,
    s3_flow_prefix,
    s3_input_data_prefix,
    s3_fs_query_output_prefix
]

In [ ]:
print(f"************************************************************************")
print(f"All objects under the following S3 prefixes will be removed permanently!")
print(f"************************************************************************")
for p in prefixes_to_delete:
    print(p)

In [ ]:
for p in prefixes_to_delete:
    !aws s3 rm s3://{p} --recursive

# Release resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

Continue with clean up as documented in [README](../README.md#clean-up).